In [33]:
import requests
import pandas as pd
import numpy as np
import requests
from newspaper import Article

# 열의 최대 너비 설정
pd.set_option('display.max_colwidth', None)

# 최대 표시 행 수 설정
pd.set_option('display.max_rows', None)

api_key = "64a1bdc693984439b9e3a23e58ea5162"

In [28]:
def get_news(api_key, start_date, end_date, query="bitcoin", language="en", page_size=10, page=1):
    base_url = "https://newsapi.org/v2/everything"
    
    parameters = {
        "q": query,
        "language": language,
        "pageSize": page_size,
        "page": page,
        "apiKey": api_key,
        "from": start_date,
        "to": end_date
    }
    
    response = requests.get(base_url, params=parameters)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [21]:
def convert_to_dataframe(news_data):
    if news_data and "articles" in news_data:
        df = pd.DataFrame(news_data['articles'])
        return df
    else:
        return pd.DataFrame()

In [29]:
# def extract_article_content(url):
#     try:
#         # Article 객체 생성
#         article = Article(url)
        
#         # 기사 내용 다운로드 및 파싱
#         article.download()
#         article.parse()
        
#         # 기사 본문 반환
#         return article.text
#     except Exception as e:
#         print(f"Error extracting content from {url}. Error: {e}")
#         return None

In [34]:
news_data = get_news(api_key)
df = convert_to_dataframe(news_data)
print(df)

                                                   source        author  \
0                     {'id': None, 'name': 'Gizmodo.com'}  Kevin Hurler   
1                     {'id': None, 'name': 'Gizmodo.com'}   Cheryl Eddy   
2                {'id': 'the-verge', 'name': 'The Verge'}     Emma Roth   
3                    {'id': None, 'name': 'Slashdot.org'}        msmash   
4            {'id': 'google-news', 'name': 'Google News'}          None   
5            {'id': 'google-news', 'name': 'Google News'}          None   
6            {'id': 'google-news', 'name': 'Google News'}          None   
7  {'id': 'business-insider', 'name': 'Business Insider'}   Matthew Fox   
8  {'id': 'business-insider', 'name': 'Business Insider'}   Matthew Fox   
9  {'id': 'business-insider', 'name': 'Business Insider'}    Phil Rosen   

                                                                                                        title  \
0                                        SEC Reportedly Asked

In [37]:
df

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'Gizmodo.com'}",Kevin Hurler,SEC Reportedly Asked Coinbase to Halt All Trading—Except for Bitcoin,"Coinbase’s CEO Brian Armstrong reportedly told the Financial Times that the U.S. Securities & Exchange Commission asked him to halt trading in the exchange with all currencies except bitcoin. According to Armstrong, the suggestion came just before the SEC sue…",https://gizmodo.com/sec-asked-coinbase-to-halt-trading-except-for-bitcoin-1850691411,"https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/76f95899df261776c441932b344183f1.jpg",2023-07-31T14:55:00Z,Coinbases CEO Brian Armstrong reportedly told the Financial Times that the U.S. Securities &amp; Exchange Commission asked him to halt trading in the exchange with all currencies except bitcoin. Acco… [+1850 chars]
1,"{'id': None, 'name': 'Gizmodo.com'}",Cheryl Eddy,Everyone's Favorite Knife-Wielding Robot Returns in Next Week's Futurama,"Futurama’s new season continues its examination of 3023 by way of 2023 with next week’s “How the West Was 1010001,” which sees the gang heading to, ahem, Doge City to cash in on the bitcoin mining craze. Like most places they visit, they encounter absurdities…",https://gizmodo.com/futurama-hulu-new-ep-3-clips-bitcoin-crypto-wild-west-1850708869,"https://i.kinja-img.com/gawker-media/image/upload/c_fill,f_auto,fl_progressive,g_center,h_675,pg_1,q_80,w_1200/11cf0e30bb3cfd27a7b0f55aa6eddfd3.jpg",2023-08-04T20:45:00Z,"Futuramas new season continues its examination of 3023 by way of 2023 with next weeks How the West Was 1010001, which sees the gang heading to, ahem, Doge City to cash in on the bitcoin mining craze.… [+2351 chars]"
2,"{'id': 'the-verge', 'name': 'The Verge'}",Emma Roth,PayPal launches PYUSD stablecoin backed by the US dollar,"PayPal has launched a stablecoin called PayPal USD, starting today and “rolling out in the coming weeks.” The new stablecoin can be used for purchases and person-to-person payments.",https://www.theverge.com/2023/8/7/23822752/paypal-pyusd-stablecoin-cryptocurrency,https://cdn.vox-cdn.com/thumbor/AzUxs8UmwIY2lOByn5LIX8geWjY=/0x0:2200x1650/1200x628/filters:focal(1100x825:1101x826)/cdn.vox-cdn.com/uploads/chorus_asset/file/24835037/PayPal_stablecoin.png,2023-08-07T14:07:51Z,PayPal launches PYUSD stablecoin backed by the US dollar\r\nPayPal launches PYUSD stablecoin backed by the US dollar\r\n / PayPal USD is built on Ethereum and is 1:1 redeemable for US dollars.\r\nPayPal is… [+1960 chars]
3,"{'id': None, 'name': 'Slashdot.org'}",msmash,First Bitcoin ETF Could Be Coming Soon as Court Rules in Favor of Grayscale Over SEC,"The U.S. Court of Appeals for the D.C. Circuit has paved the way for bitcoin exchange-traded funds. From a report: On Tuesday, the court sided with Grayscale in a lawsuit against the Securities and Exchange Commission which had denied the company's applicatio…",https://slashdot.org/story/23/08/29/1816233/first-bitcoin-etf-could-be-coming-soon-as-court-rules-in-favor-of-grayscale-over-sec,https://a.fsdn.com/sd/topics/bitcoin_64.png,2023-08-29T19:20:00Z,"On Tuesday, the court sided with Grayscale in a lawsuit against the Securities and Exchange Commission which had denied the company's application to convert the Grayscale Bitcoin Trust to an ETF. The… [+1393 chars]"
4,"{'id': 'google-news', 'name': 'Google News'}",None,"Before suing Coinbase, SEC asked it to trade only in bitcoin -FT - Reuters","Before suing Coinbase, SEC asked it to trade only in bitcoin -FT Reuters",https://consent.google.com/ml?continue=https://news.google.com/rss/articles/CBMifmh0dHBzOi8vd3d3LnJldXRlcnMuY29tL3RlY2hub2xvZ3kvc2VjLWFza2VkLWNvaW5iYXNlLXN0b3AtdHJhZGluZy1jcnlwdG9jdXJyZW5jaWVzLW90aGVyLXRoYW4tYml0Y29pbi1wcmlvci1zdWluZy0yMDIzLTA3LTMxL9IBAA?oc%3D5&gl=FR&hl=en-US&cm=2&pc=n&src=1,None,2023-07-31T05:06:42Z,"We use cookies and data to<ul><li>Deliver and maintain Google services</

In [36]:
df['content'][0]

'Coinbases CEO Brian Armstrong reportedly told the Financial Times that the U.S. Securities &amp; Exchange Commission asked him to halt trading in the exchange with all currencies except bitcoin. Acco… [+1850 chars]'

In [30]:
# # df는 이전에 생성한 뉴스 데이터 프레임입니다.
# df['content'] = df['url'].apply(extract_article_content)
# print(df[['title', 'content']])

                                               title  \
0  SEC Reportedly Asked Coinbase to Halt All Trad...   
1  Everyone's Favorite Knife-Wielding Robot Retur...   
2  PayPal launches PYUSD stablecoin backed by the...   
3  First Bitcoin ETF Could Be Coming Soon as Cour...   
4  Before suing Coinbase, SEC asked it to trade o...   
5  Binance Australia customers seen selling bitco...   
6  Crypto stocks dip after bitcoin slumps to six-...   
7  Bitcoin plunges more than 10% after Elon Musk'...   
8  Bitcoin could drop to the $20,000 range if a k...   
9  Bitcoin is headed to $180,000 if regulators ap...   

                                             content  
0  Coinbase’s CEO Brian Armstrong reportedly told...  
1  Futurama’s new season continues its examinatio...  
2  PayPal is launching its own stablecoin: PayPal...  
3  The U.S. Court of Appeals for the D.C. Circuit...  
4  A smartphone with displayed Coinbase logo and ...  
5  The Binance logo is seen in this illustration ... 